In [2]:
using Distributed 
addprocs(2)
@everywhere using DelimitedFiles
@everywhere using Random, Distributions;  Random.seed!();
# @everywhere using PyPlot
@everywhere using Distributed

In [3]:
@everywhere using Statistics
@everywhere using SpecialFunctions
@everywhere using QuadGK
@everywhere using DataStructures
@everywhere using DelimitedFiles
@everywhere using Roots

In [4]:
#SIk
@everywhere function SIk(x,parms)
    (S,I) = x
    (μ,β,κ,γ,N) = parms
    infection = β*(N-I)*I/N + κ*(N-I)
    recov_death = (γ+μ)*I
#     death_INF = μ*I
    event_weight = [infection recov_death]
    event_shift = [[0 1]; [0 -1]]
    return event_weight, event_shift
end

In [8]:
@everywhere function GSP(F::Function,state,parms,endcondition,time_end)
    time_points = collect(0:2:floor(time_end))
    I_vec_out = zeros(length(time_points))
    I_vec_out[1] = state[2]
#     fin = length(time_points)
    t = 0 #initial time is zero
    a = []    
    enact_event = zeros(0)
#     j = 2 # for progressing time_points

####################################    
#### Disease Progression Record ####
#     t_vec = zeros(0)
#     append!(t_vec,0)
#     I_vec = zeros(0)
#     append!(I_vec,state[2])
#### Disease Progression Record ####    
####################################
 
    while t<(time_points[end] +2)
        
        # to build time interval
        t1 = t
        
        ###############
        ## time step ##
        ###############
        a = F(state,parms)

        cumul_a = cumsum(a[1][1,:])
        num_o_events = length(a[1][1,:])
        a0 = cumul_a[num_o_events]
        #random numbers
        r1 = rand()
        r2 = rand()
        dtau = (1/a0)*log(1/r1)
        t += dtau
        
        ################################
        ### Append the output vector ###
        ################################

        timeloc =  findall(x->(x>t1 && x<=t), time_points)
        I_vec_out[timeloc] = repeat([state[2]],inner=1,outer=length(timeloc))
        

        ##############################
        ### determine event ##########
        ##############################
        marker = r2*a0
        i=1
        while i<=num_o_events
            if marker<=cumul_a[i]
                enact_event = i
                i = num_o_events+1
            end
            i+=1
        end


        #######################
        ### change the state ##
        #######################
        state = state + (a[2][enact_event,:])' #these are the state alterations defined by the function


    end #end while

#     return t
#     plt.plot(t_vec,I_vec)
    return time_points, I_vec_out
end 

In [9]:
###################
### run GSP sim ###
###################
@everywhere using Printf
@everywhere function run_GSP(kval_u,Nval_u)
    
    # set parameters find equilibrium
    param=[5e-5 0.5 kval_u 1 Nval_u]

    function i_equation_u(I,param)
        (μ,β,κ,γ,N) = param
        return (κ+β*I)*μ - (μ+κ+β*I)*(μ+γ)*I
    end
    function i_equation(I)
        return i_equation_u(I,param)
    end
    ieq = fzero(i_equation,1)
    function sequilibrium(param)
        (μ,β,κ,γ,N) = param
        return μ / (μ+κ+β*ieq)
    end
    seq = sequilibrium(param)

    # Make initial conditions
    (μ,β,κ,γ,N) = param
    u0 = [floor(seq*param[5]) floor(ieq*param[5])]

    # run the Gillespie simulation
    simdat = GSP(SIk,u0,param,20000,20000); 
    n_len = length(simdat[1])
    kap_vec = zeros(n_len)
    kap_vec[1] = κ
    N_vec = zeros(n_len)
    N_vec[1] = N
    
    # Make strings for file names
    ku = @sprintf("%.3E", κ)
    ku = replace(ku, "." => "p")
    ku = replace(ku, "-" => "m")
    ku = replace(ku, "+" => "")
    
    Nu = @sprintf("%.3E", N)
    Nu = replace(Nu, "." => "p")
    Nu = replace(Nu, "-" => "m")
    Nu = replace(Nu, "+" => "")
    
    simdat_util = [simdat[1] simdat[2] kap_vec N_vec] 
    flname = "SIk_keq_"*ku*"_Neq_"*Nu
    writedlm(flname,  simdat_util, ',') 

    return nothing
#     return simdat
end

In [10]:
# make sample list of kappa values
kappa_list_util = 10.0 .^(range(-7,stop=-4,length=30))
kappa_list = zeros(0)

# make sample list of N values
N_list_util = 10.0 .^(range(2,stop=5,length=30))
N_list = zeros(0)

for i in kappa_list_util
    for j in N_list_util
        # make lists for pmap
        append!(kappa_list, i)
        append!(N_list, j)
    end
end
pmap(run_GSP,kappa_list,N_list)

900-element Array{Nothing,1}:
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 ⋮      
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing

In [ ]:
@everywhere function GSP(F::Function,state,parms,endcondition,time_end)
    time_points = collect(0:2:floor(time_end))
    fin = length(time_points)
    t = 0 #initial time is zero
    a = []    
    enact_event = zeros(0)
    j = 2 # for progressing time_points

####################################    
#### Disease Progression Record ####
    t_vec = zeros(0)
    append!(t_vec,0)
    I_vec = zeros(0)
    append!(I_vec,state[2])
#### Disease Progression Record ####    
####################################
 
    while t<time_points[fin]
        
        a = F(state,parms)

        cumul_a = cumsum(a[1][1,:])
        num_o_events = length(a[1][1,:])
        a0 = cumul_a[num_o_events]
        #random numbers
        r1 = rand()
        r2 = rand()

         

        ##############################
        ### determine event ##########
        ##############################
        marker = r2*a0
        i=1
        while i<=num_o_events
            if marker<=cumul_a[i]
                enact_event = i
                i = num_o_events+1
            end
            i+=1
        end
        

        ###############
        ## time step ##
        ###############
        dtau = (1/a0)*log(1/r1)
        t += dtau

        #######################
        ### change the state ##
        #######################
        state = state + (a[2][enact_event,:])' #these are the state alterations defined by the function
       
        ################################
        ### Append the output vector ###
        ################################

        if (t>time_points[j] && (t<time_points[j]+2.0)) 
            t = time_points[j]
            append!(I_vec,state[2])
            append!(t_vec,t)
            j += 1
        elseif t>(time_points[j]+2)
            indx_tps = floor( (t-time_points[j])  /2)

            for tps in collect(0:1:indx_tps)
                if (j+tps)<fin
                    jtps = Int(floor(j+tps))
                    t = time_points[jtps]
                    append!(I_vec,state[2])
                    append!(t_vec,t)
                    j+=1
                end #if
            end #for

        end #if 


    end #end while

#     return t
#     plt.plot(t_vec,I_vec)
    return t_vec, I_vec
end 

In [ ]:
time_end = 50
time_points = collect(0:2:floor(time_end))
timeloc = findall(x->(x>46 && x<50),time_points)
time_points[timeloc]
time_points[end]
I_vec = zeros(length(time_points));

t1 = 3
t = 9.25
Ival = 7

timeloc =  findall(x->(x>t1 && x<=t), time_points)
I_vec[timeloc] = repeat([Ival],inner=1,outer=length(timeloc))
I_vec

In [ ]:
#SIk
@everywhere function SIk(x,parms)
    (S,I) = x
    (μ,β,κ,γ,N) = parms
#     I = convert(AbstractFloat, I[1])
    birth = μ*N
    death = μ*S
    infection = β*(N-I)*I/N + κ*(N-I)
    recovery = γ*I
    death_INF = μ*I
    event_weight = [birth death infection recovery death_INF]
    event_shift = [[1 0]; [-1 0]; [-1 1]; [1 -1]; [0 -1]]
    return event_weight, event_shift
end

In [ ]:
# time_end = 100
# time_points = collect(0:2:floor(time_end))
# fin = length(time_points)
# t = 0 #initial time is zero
# a = []    
# enact_event = zeros(0)
# j = 2 # for progressing time_points
# # println(time_points)
# t=20
# print(t)

# 🐢 = 20

# if t > time_points[🐢] && t < time_points[🐢+1]
#     print("hi")
# end

# for 🐢 in 1:(length(time_points)-1)
#     if (t > time_points[🐢] & t<time_points[🐢+1])
#         println(time_points[🐢])
#         println(time_points[🐢+1])        
#         println(🐢)
#     end
# end
